In [28]:
from bs4 import BeautifulSoup
from requests import get 
import pandas as pd 
import datetime as dt

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

permitNums = []
urls = []
dates = []
descs = []
statuses = []
addresses = []

In [29]:
def openDriver():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get('https://accela.maricopa.gov/CitizenAccessMCOSS/Cap/CapHome.aspx?module=PnD&TabName=PnD')
    return driver

def searchNewSfm(driver):
    # Select new residential builds
    Select(driver.find_element(By.XPATH, '//*[@id="ctl00_PlaceHolderMain_generalSearchForm_ddlGSPermitType"]')).select_by_visible_text('Residential New Single Family Residence')
    # Select beginning of the year date
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="ctl00_PlaceHolderMain_generalSearchForm_txtGSStartDate"]').send_keys('01012023')
    # Click search
    driver.find_element(By.XPATH, '//*[@id="ctl00_PlaceHolderMain_btnNewSearch"]').click()
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList")))
    soup = getPage(driver)
    return soup

def getPage(driver):
    # Get HTML
    page = driver.page_source
    soup = BeautifulSoup(page)
    return soup

def getPermitInfo(soup):
    # Get permit info
    table = soup.find(class_='ACA_GridView ACA_Grid_Caption')
    print(table.find_all('tr')[0].text.strip())
    permits = table.find_all('tr')[3:-3]
    for permit in permits:
        try:
            urls.append(f'https://accela.maricopa.gov/{permit.find_all("a")[0]["href"]}')
        except:
            urls.append(None)
        spans = permit.find_all('span')
        date = spans[0].text
        dates.append(dt.date(month=int(date[:2]), day=int(date[3:5]), year=int(date[-4:])))
        permitNums.append(spans[1].text)
        descs.append(spans[3].text.replace('\n',''))
        statuses.append(spans[-3].text)
        addresses.append(spans[-1].text)

def clickNext(driver):
    driver.find_element(By.PARTIAL_LINK_TEXT, "Next").click()
#     //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[14]/a
# //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[13]/a
# //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[13]/a
# //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[14]/a
# //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[14]/a
# //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[14]/a
# //*[@id="ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList"]/tbody/tr[13]/td/table/tbody/tr/td[14]/a
    # element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList")))
    # <a class="aca_simple_text font11px" href="javascript:__doPostBack('ctl00$PlaceHolderMain$dgvPermitList$gdvPermitList$ctl13$ctl14','');var p = new ProcessLoading();p.showLoading(false);">Next &gt;</a>
    

In [30]:
driver = openDriver()
soup = searchNewSfm(driver)
getPermitInfo(soup)

page = 1
while True:
    page += 1
    # print(page)
    try:
        clickNext(driver)
        time.sleep(8)
        # element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "aca_simple_text font11px")))
        soup = getPage(driver)
        getPermitInfo(soup)
    except Exception as e:
        if page < 100:
            True
        else:
            break

Showing 1-10 of 100+
Showing 11-20 of 100+
Showing 21-30 of 100+
Showing 31-40 of 100+
Showing 41-50 of 100+
Showing 51-60 of 100+
Showing 61-70 of 100+
Showing 71-80 of 100+
Showing 81-90 of 100+
Showing 91-100 of 100+
Showing 101-110 of 200+
Showing 111-120 of 200+
Showing 121-130 of 200+
Showing 131-140 of 200+
Showing 141-150 of 200+
Showing 151-160 of 200+
Showing 151-160 of 200+
Showing 171-180 of 200+
Showing 181-190 of 200+
Showing 191-200 of 200+
Showing 201-210 of 300+
Showing 211-220 of 300+
Showing 221-230 of 300+
Showing 231-240 of 300+
Showing 241-250 of 300+
Showing 251-260 of 300+
Showing 261-270 of 300+
Showing 271-280 of 300+
Showing 281-290 of 300+
Showing 291-300 of 300+
Showing 301-310 of 400+
Showing 311-320 of 400+
Showing 321-330 of 400+
Showing 331-340 of 400+
Showing 341-350 of 400+
Showing 351-360 of 400+
Showing 361-370 of 400+
Showing 371-380 of 400+
Showing 381-390 of 400+
Showing 391-400 of 400+
Showing 401-410 of 500+
Showing 401-410 of 500+
Showing 411-

KeyboardInterrupt: 

In [36]:
df = pd.DataFrame(dict(zip(['permit', 'url', 'date', 'desc', 'status', 'address'],[permitNums, urls, dates, descs, statuses, addresses])))

In [37]:
df.drop_duplicates(inplace=True)
df

,permit,url,date,desc,status,address
0,B202307675,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-07-21,"New Custom Residence, with basement. Wood fram...",Initialized,"2805 W Colt Rd, CHANDLER, AZ 85224 United States"
1,B202307686,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-07-21,NEW SINGLE FAMILY HOME (SINGH),Initialized,"0 TBD, TONOPAH AZ 85354 United States"
2,B202307688,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-07-21,NEW SINGLE FAMILY HOME (SINGH)16,Initialized,"0 TBD, TONOPAH AZ 85354 United States"
3,B202307694,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-07-21,NEW SINGLE FAMILY HOME (ACOSTA),Initialized,"0 TBD, PEORIA AZ 85383 United States"
4,B202307697,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-07-21,NEW SINGLE FAMILY RESIDENCE (ROMERO),Initialized,"1016 N 371 St, TONOpah AZ 85354 United States"
...,...,...,...,...,...,...
498,B202300090,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-01-04,CUSTOM STEEL-FRAMED SINGLE FAMILY RESIDENCE /...,Terminate,"1217 E YUCCA Ln, NEW RIVER AZ 85087"
499,B202300094,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-01-04,New residential home,Terminate,"750 Shamrock Ln, Cle Elum WA 98922 United States"
500,B202300105,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-01-04,CUSTOM SINGLE FAMILY RESIDENCE / 565 LF X 6 FT...,QC OTC,"31229 N 159TH Ave, PEORIA AZ 85383"
501,B202300114,https://accela.maricopa.gov//CitizenAccessMCOS...,2023-01-04,CUSTOM SINGLE FAMILY RESIDENCE // INCLUDING B...,Issued,"3323 W ELLIOT Rd, LAVEEN AZ 85339"


In [35]:
# manually adding 161-179 & 511-520
soup = getPage(driver)
getPermitInfo(soup)

Showing 511-520 of 529


In [38]:
df.to_csv('Maricopa2023sfr.csv', index=False)